# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Male, VISl, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 824


,waveform_velocity_below,amplitude_cutoff,waveform_repolarization_slope,snr,firing_rate,waveform_duration,presence_ratio,isi_violations,cumulative_drift,waveform_spread,max_drift,waveform_PT_ratio,nn_hit_rate,waveform_recovery_slope,waveform_velocity_above,isolation_distance,L_ratio,peak_channel_id,d_prime,cluster_id,waveform_amplitude,waveform_halfwidth,local_index_unit,silhouette_score,nn_miss_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
915956356,NaN,0.054706,0.632977,2.605408,2.193113,0.549414,0.99,0.430427,263.01,60.0,18.37,0.512847,0.637363,-0.106051,-0.451304,18.136302,0.061345,850229419,2.936851,14,214.954545,0.233501,14,0.108867,0.000673,NaN,1100.0,1.699291,0.776937,0.920000,0.650000,1.452569,0.592722,0.047321,0.017619,0.039495,53.636,0.204814,0.0,0.04,15.0,0.332813,-0.142857,-0.077160,NaN,0.595238,315.0,60.0,0.106585,0.749752,0.770016,NaN,0.029437,-15.455,4992,0.75,2.247928,1.438807,2.613584,1.564701,2.218146,NaN,0.0575,False,False,False,False,False,-0.046051,False,0.114104,0.241255,0.056767,0.416234,0.067618,40,27,733744647,3,8.0,grey,NaN,NaN,NaN,probeB,See electrode locations,29999.915391,1249.996475,True
915956349,NaN,0.096378,0.553393,2.684636,16.192413,0.508208,0.99,0.120715,169.29,40.0,14.87,0.424620,0.874667,-0.086022,0.206030,54.635515,0.010406,850229419,2.973959,13,177.115380,0.192295,13,0.136901,0.021636,NaN,600.0,5.764529,0.877288,1.189189,1.268914,2.335358,0.288541,0.015031,0.040988,0.087448,58.333,0.960864,0.0,0.02,15.0,-0.032581,0.143963,0.006477,NaN,0.137202,180.0,90.0,0.854594,0.416630,0.955454,NaN,0.294999,-20.000,4944,0.00,16.046763,7.247322,17.963269,9.209478,15.128685,NaN,0.0705,False,False,False,False,False,0.323068,False,0.095038,0.166625,0.028184,0.073108,0.029319,40,27,733744647,3,8.0,grey,NaN,NaN,NaN,probeB,See electrode locations,29999.915391,1249.996475,True
915956345,NaN,0.001703,0.781296,3.362198,12.698430,0.521943,0.99,0.044936,129.36,30.0,11.56,0.500520,0.930000,-0.153127,0.343384,47.805714,0.008281,850229419,3.559911,12,251.275830,0.192295,12,0.138827,0.004956,NaN,600.0,4.236128,1.643568,1.817460,0.990123,3.421208,0.268001,0.038387,0.040152,0.085168,40.000,0.123119,0.0,0.04,8.0,0.222727,0.006618,-0.096483,-0.081081,0.431100,225.0,60.0,0.178474,0.982081,0.509995,0.545245,0.020251,-25.000,5010,0.00,11.973721,7.860148,15.618365,12.267608,10.896224,NaN,0.0725,False,False,False,False,False,0.179102,False,0.086342,0.337854,0.045184,0.302351,0.062177,40,27,733744647,3,8.0,grey,NaN,NaN,NaN,probeB,See electrode locations,29999.915391,1249.996475,True
915956340,NaN,0.000881,0.704522,3.357908,9.660554,0.563149,0.99,0.006825,160.24,30.0,7.79,0.439372,0.987654,-0.104196,0.000000,59.613182,0.000410,850229419,5.519024,11,247.254345,0.206030,11,0.197458,0.000903,NaN,600.0,13.657938,1.428046,3.537971,4.437255,4.910479,0.419164,0.094477,0.145318,0.075828,25.000,0.209387,0.0,0.02,8.0,0.355659,0.207353,0.397513,-0.054545,0.651261,270.0,90.0,0.209249,0.435825,0.028048,0.931928,0.00075

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 22


,waveform_velocity_below,amplitude_cutoff,waveform_repolarization_slope,snr,firing_rate,waveform_duration,presence_ratio,isi_violations,cumulative_drift,waveform_spread,max_drift,waveform_PT_ratio,nn_hit_rate,waveform_recovery_slope,waveform_velocity_above,isolation_distance,L_ratio,peak_channel_id,d_prime,cluster_id,waveform_amplitude,waveform_halfwidth,local_index_unit,silhouette_score,nn_miss_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
915963371,-0.755444,0.002183,0.034464,0.937302,0.816827,1.304858,0.97,0.000000,96.99,60.0,22.96,3.105986,0.923457,-0.040724,0.000000,48.615511,0.006257,850232741,5.329276,792,132.544425,NaN,772,0.160696,0.000203,NaN,100.0,2.624242,0.365980,0.833333,0.933333,0.862222,1.003737,0.038593,0.165635,0.200268,50.000,1.128866,0.504,0.02,1.0,0.904762,0.325163,0.606061,0.771429,0.222222,315.0,0.0,0.026350,0.005173,0.000140,0.136444,0.666460,-30.000,4919,0.50,0.326672,13.508796,2.281764,0.315791,0.414986,NaN,0.0715,False,False,False,False,False,0.333644,False,0.333139,0.010595,0.090628,0.456618,0.380962,2640,27,733744651,263,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.916678,1249.996528,True
915962742,-1.716918,0.093904,0.033417,0.592976,4.537270,1.263652,0.99,0.061884,367.51,70.0,35.14,10.334414,0.800000,-0.000911,1.236181,44.908559,0.018769,850232761,2.645302,562,86.294130,NaN,545,0.055770,0.003449,NaN,2600.0,4.909524,1.630476,1.512603,5.055399,1.914286,1.105674,0.134628,0.145079,0.057782,50.769,0.622050,0.000,0.32,8.0,-0.060185,0.747243,-0.325926,0.441558,0.200603,270.0,90.0,0.927898,0.042222,0.181101,0.475528,0.542147,-20.385,5016,0.25,6.609396,1.518740,3.707783,5.396958,3.011482,NaN,0.1345,False,False,False,False,False,0.012169,False,0.087584,0.366516,0.057239,0.218702,0.078992,2740,11,733744651,273,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.916678,1249.996528,True
915962725,0.000000,0.064785,0.087893,0.890068,15.336706,1.181240,0.99,0.023189,137.34,90.0,27.60,1.294831,0.986000,-0.001782,0.412060,79.570574,0.000697,850232769,5.347278,556,110.585475,0.521943,539,0.100600,0.001896,NaN,300.0,12.604974,1.328151,0.862979,2.093694,1.698497,0.276508,0.024273,0.136518,0.041388,60.000,0.230440,0.000,0.02,15.0,0.563492,0.388592,0.280973,0.435737,0.356061,315.0,150.0,0.008608,0.014367,0.000422,0.155564,0.002056,50.000,4919,0.25,16.157247,43.217489,19.709689,12.052694,17.957654,NaN,0.0895,False,False,False,False,False,0.306915,False,0.042577,0.033593,0.024422,0.064020,0.027896,2780,11,733744651,277,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.916678,1249.996528,True
915962810,0.686767,0.000569,0.040768,0.849503,3.271240,1.291122,0.99,0.085570,167.04,70.0,25.85,2.614304,0.847690,-0.009698,0.686767,57.312356,0.007966,850232783,4.329509,586,152.807850,NaN,569,0.153492,0.001884,NaN,200.0,7.571521,1.166061,2.516154,2.660606,2.800031,0.610335,0.040833,0.570771,0.452826,65.000,7.992815,0.000,0.02,1.0,0.190678,0.423529,0.501489,-0.565789,0.452491,90

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
915963371     0.414986
915962742     3.011482
915962725    17.957654
915962810     2.383341
915962819     6.064263
915962825    12.664745
915962815     5.375506
915962841     2.793664
915962847     0.949873
915962868     6.060933
915962863     1.457448
915962880     3.943370
915962861     1.747872
915962935    11.492392
915962933     6.010975
915962950     0.209825
915962964     7.476416
915962976     0.800664
915963000     4.255109
915963076     0.395669
915963123     1.471437
915963135     0.907242
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

4.447493897713799


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
915963371     0.326672
915962742     6.609396
915962725    16.157247
915962810     3.955652
915962819     5.664969
915962825    16.966225
915962815    13.282534
915962841     1.227012
915962847     3.104711
915962868     4.815622
915962863     1.544122
915962880     5.434440
915962861     3.743182
915962935    11.859519
915962933     2.101855
915962950     0.290021
915962964     5.280399
915962976     0.818008
915963000     2.705269
915963076     0.671935
915963123     2.587348
915963135     3.277343
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

5.110158199755199


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])